# Multiple parallel input and Multi-Step output-Gru
## Developer: mehdi sahraei
### Importing


In [3]:
from numpy import array
from numpy import hstack
from keras.models import  Sequential
from keras.layers import GRU, Dense, TimeDistributed, RepeatVector, Dropout

### method

In [4]:
def split_sequence(sequence, n_step_in, n_step_out):
   x,y = list(), list() #x is data and y is label
   for i in range(len(sequence)):
       end_ix = i + n_step_in #timestep
       out_end_ix = end_ix + n_step_out # n-output
       if out_end_ix > len(sequence): #In this -if- we want if data be beyond len(sequence) , the command will not continue and break.
          break
       seq_x, seq_y = sequence[i:end_ix,:] , sequence[end_ix:out_end_ix,:]   
       x.append(seq_x)
       y.append(seq_y)
   return array(x) , array(y)

- making data x,y

In [5]:
#n_step_in: timestep or look_up
n_step_in, n_step_out = 3 , 3   

seq1 = array([10,20,30,40,50,60,70,80,90])
seq2 = array([10,20,30,40,50,60,70,80,90])
seq3 = array([10,20,30,40,50,60,70,80,90])   
   
#The input must be three-dimensional or three-channel, that's why we reshape it
seq1 = seq1.reshape((len(seq1),1))
seq2 = seq2.reshape((len(seq2),1))
seq3 = seq3.reshape((len(seq3),1))
   
dataset = hstack((seq1,seq2,seq3))      


x,y = split_sequence(dataset, n_step_in, n_step_out)
print(x.shape,y.shape)   
print('-'*75) #print -----
   
for i in range(len(x)):   
   print(x[i],y[i])


(4, 3, 3) (4, 3, 3)
---------------------------------------------------------------------------
[[10 10 10]
 [20 20 20]
 [30 30 30]] [[40 40 40]
 [50 50 50]
 [60 60 60]]
[[20 20 20]
 [30 30 30]
 [40 40 40]] [[50 50 50]
 [60 60 60]
 [70 70 70]]
[[30 30 30]
 [40 40 40]
 [50 50 50]] [[60 60 60]
 [70 70 70]
 [80 80 80]]
[[40 40 40]
 [50 50 50]
 [60 60 60]] [[70 70 70]
 [80 80 80]
 [90 90 90]]


### training model

In [7]:
n_features = x.shape[2]
n_features

3

In [17]:
model = Sequential()
model.add(GRU(units=100,activation = 'tanh',input_shape=(n_step_in,n_features))) # tanh is best activation in gru but relu also is used
model.add(RepeatVector(n_step_out))   
model.add(GRU(units=100,activation = 'tanh',return_sequences=True))   
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='RMSprop',loss='mse')  #also adam is used

In [10]:
model.fit(x,y,epochs=200,verbose=1)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 4385.7734
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 4292.6641
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 4201.0288
Epoch 4/200
1/1 [==============================] - 0s 12ms/step - loss: 4110.6348
Epoch 5/200
1/1 [==============================] - 0s 14ms/step - loss: 4020.3484
Epoch 6/200
1/1 [==============================] - 0s 15ms/step - loss: 3927.8040
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 3831.3535
Epoch 8/200
1/1 [==============================] - 0s 12ms/step - loss: 3731.0051
Epoch 9/200
1/1 [==============================] - 0s 10ms/step - loss: 3626.8098
Epoch 10/200
1/1 [==============================] - 0s 12ms/step - loss: 3520.0803
Epoch 11/200
1/1 [==============================] - 0s 10ms/step - loss: 3410.7366
Epoch 12/200
1/1 [==============================] - 0s 11ms/step - loss: 3293.2942
Epoch 13/200
1/

1/1 [==============================] - 0s 15ms/step - loss: 2.0689
Epoch 102/200
1/1 [==============================] - 0s 16ms/step - loss: 1.9211
Epoch 103/200
1/1 [==============================] - 0s 15ms/step - loss: 1.7849
Epoch 104/200
1/1 [==============================] - 0s 16ms/step - loss: 1.6642
Epoch 105/200
1/1 [==============================] - 0s 14ms/step - loss: 1.6127
Epoch 106/200
1/1 [==============================] - 0s 15ms/step - loss: 1.5496
Epoch 107/200
1/1 [==============================] - 0s 15ms/step - loss: 1.5384
Epoch 108/200
1/1 [==============================] - 0s 15ms/step - loss: 1.5028
Epoch 109/200
1/1 [==============================] - 0s 15ms/step - loss: 1.4799
Epoch 110/200
1/1 [==============================] - 0s 14ms/step - loss: 1.4431
Epoch 111/200
1/1 [==============================] - 0s 14ms/step - loss: 1.3928
Epoch 112/200
1/1 [==============================] - 0s 15ms/step - loss: 1.3502
Epoch 113/200
1/1 [=======================

### testing

In [11]:
x_input = array([[10,10,10],[20,20,20],[30,30,30]])
x_input = x_input.reshape(1,n_step_in,n_features)
x_input

array([[[10, 10, 10],
        [20, 20, 20],
        [30, 30, 30]]])

In [12]:
yhat = model.predict(x_input,verbose=0)
print(yhat,'\n')
# print(y[0])

[[[39.689754 39.22587  39.110672]
  [50.40922  50.21673  50.163116]
  [60.061512 59.730053 59.888596]]] 

